In [58]:
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import openai
from openai import OpenAI

**Question 1**

In [3]:
! curl localhost:9200

{
  "name" : "1775a78d5c65",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "9_GHgz6JRieVZV_m3WHMiw",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

**Question 2**

In [6]:
es_client = Elasticsearch('http://localhost:9200')

In [7]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

**Question 3**

In [14]:
# Question 3
query = "How do execute a command on a Kubernetes pod?"
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [15]:
response = es_client.search(index=index_name, body=search_query)

In [17]:
response['hits']['hits'][0]['_score']

44.50556

**Question 4**

In [18]:
#Question 4
query = "How do copy a file to a Docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [19]:
response = es_client.search(index=index_name, body=search_query)

In [25]:
response['hits']['hits'][2]

{'_index': 'course-questions',
 '_id': 'sl4Wc5cBo3nqRrBLZAXH',
 '_score': 59.812744,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

**Question 5**

In [42]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
for r in response['hits']['hits']:
    q = r['_source']['question']
    t = r['_source']['text']
    context = context + context_template.format(question=q, text=t) + "\n\n"      

In [43]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [44]:
prompt = prompt_template.format(question=query, context=context)

In [46]:
len(prompt)

1448

**Question 6**

In [47]:
# Q6
import tiktoken

In [48]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [50]:
tokenized_prompt = encoding.encode(prompt)
len(tokenized_prompt)

321

In [53]:
encoding.decode_single_token_bytes(tokenized_prompt[0])

b"You're"

**Question 7**

In [54]:
# Q7
client = OpenAI()

In [55]:
def llm(prompt, client):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [56]:
llm(prompt, client)

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows:\n\n```\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```'

In [57]:
# Q8
#Input: $0.005 / 1K tokens
input_cost = 0.005
#Output: $0.015 / 1K tokens
ouput_cost = 0.015 
# Input tokens per request : 150
# Output tokens per request = 250
# Cost for 1000 requests : 
cost = input_cost * 150 + ouput_cost * 250
print(cost)

4.5
